In [1]:
%run cnn_text_classifier.ipynb

In [2]:
SEQ_LENGTH = 128
FINETUNING_LEARNING_RATE = 1e-3

In [3]:
import os
import re
import math
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from datasets import Dataset

ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
batch_sentences = [
    'This is positive',
    'This is also positive',
    'a test and',
    'a test haha',
    'a test no',
    'a test oh no',
    'a test but',
    'a test cool',
]

label = np.array([
    [0,1],
    [0,1],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
])

In [ ]:
batch = tokenizer(
    batch_sentences, 
    padding=True, 
    truncation=False, 
    return_tensors="tf")

input_ids = tf.keras.utils.pad_sequences(
    batch['input_ids'],
    maxlen=SEQ_LENGTH,
    dtype='int32',
    padding='post',
    truncating='post',
    value=0.0
    )

In [ ]:
class TextSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

In [ ]:
data_sequence = TextSequence(
    input_ids,
    label,
    batch_size = 2
    )

# training model 

In [ ]:
model = cnn_text_single_label_classifier(
    num_label = 2,
    seq_length = SEQ_LENGTH,
    )

In [ ]:
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=FINETUNING_LEARNING_RATE),
    metrics=[    
        tf.keras.metrics.AUC(name = 'au-roc',),
        tf.keras.metrics.AUC(curve='PR', name = 'au-prc',),
        tf.keras.metrics.PrecisionAtRecall(recall = 0.9, name = 'precision'),    
    ],
)

In [ ]:
model.fit(
    data_sequence,
    epochs = 200,
    batch_size = 8,
    shuffle= True,
    )

# end